## Análisis Exploratorio de Datos - Propensión de Compra

## Importación de librerías.

In [142]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import numpy as np
import re
from tabulate import tabulate
from sklearn.cluster import KMeans
import os
from urllib.parse import urlparse
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
    precision_score,
    recall_score,
)

## Importación e información del dataframe

In [143]:
# Ruta de exportación
csv_path = r"C:\Users\Claudia\OneDrive - Universidad Alfonso X el Sabio\Documentos\GitHub\Caso01_Propensity\data.csv"

# Cargar los datos en un DataFrame
df = pd.read_csv(csv_path, delimiter=',')

In [144]:
# Número de filas y columnas
print(df.shape)  

(58049, 24)


In [145]:
# Primeras filas del DataFrame
df.head()

,CODE,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,M,Empresa,...,Medio-Bajo,NO DATA,Averia muy grave,SI,2892,0,False,2,18,0
1,CLI2,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,F,Empresa,...,Medio-Bajo,SI,No,NO,1376,7187,False,2,53,0
2,CLI3,A,TIPO1,FUEL 1,Baja,M,Otros,CASADO,M,Empresa,...,Medio,NO DATA,No,NO,1376,0,True,4,21,3
3,CLI4,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Empresa,...,Medio,SI,Averia muy grave,SI,2015,7256,True,4,48,5
4,CLI5,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Funcionario,...,Alto,NO DATA,No,NO,1818,0,True,3,21,3


In [146]:

df.dtypes

CODE               object
PRODUCTO           object
TIPO_CARROCERIA    object
COMBUSTIBLE        object
Potencia           object
TRANS              object
FORMA_PAGO         object
ESTADO_CIVIL       object
GENERO             object
OcupaciOn          object
PROVINCIA          object
Campanna1          object
Campanna2          object
Campanna3          object
Zona_Renta         object
REV_Garantia       object
Averia_grave       object
QUEJA_CAC          object
COSTE_VENTA         int64
km_anno             int64
Mas_1_coche          bool
Revisiones          int64
Edad_Cliente        int64
Tiempo              int64
dtype: object

In [147]:
df.columns

Index(['CODE', 'PRODUCTO', 'TIPO_CARROCERIA', 'COMBUSTIBLE', 'Potencia',
       'TRANS', 'FORMA_PAGO', 'ESTADO_CIVIL', 'GENERO', 'OcupaciOn',
       'PROVINCIA', 'Campanna1', 'Campanna2', 'Campanna3', 'Zona_Renta',
       'REV_Garantia', 'Averia_grave', 'QUEJA_CAC', 'COSTE_VENTA', 'km_anno',
       'Mas_1_coche', 'Revisiones', 'Edad_Cliente', 'Tiempo'],
      dtype='object')

### Descripción de las Variables del Dataset

| **Variable**      | **Descripción** |
|------------------|------------------------------------------------|
| `CODE`          | Código único del cliente. |
| `PRODUCTO`      | Modelo de coche adquirido. |
| `TIPO_CARROCERIA` | Tipo de carrocería del coche (SUV, Sedán, etc.). |
| `COMBUSTIBLE`   | Tipo de combustible utilizado (Gasolina, Diésel, etc.). |
| `Potencia`      | Potencia del coche en caballos de fuerza. |
| `TRANS`         | Tipo de transmisión (Manual o Automática). |
| `FORMA_PAGO`    | Método de pago (Financiado o Contado). |
| `ESTADO_CIVIL`  | Estado civil del cliente al momento de la compra. |
| `GENERO`        | Género del cliente (Masculino o Femenino). |
| `OcupaciOn`     | Ocupación del cliente en el momento de la compra. |
| `PROVINCIA`     | Provincia de residencia del cliente. |
| `Campanna1`     | Si el cliente recibió un descuento por financiar (Sí/No). |
| `Campanna2`     | Si el cliente recibió un descuento por adquirir un seguro (Sí/No). |
| `Campanna3`     | Si el cliente recibió un descuento por extender la garantía (Sí/No). |
| `Zona_Renta`    | Nivel de renta promedio en la zona de residencia del cliente. |
| `REV_Garantia`  | Si el coche sigue en garantía. |
| `Averia_grave`  | Si el coche ha tenido una avería grave y de qué tipo. |
| `QUEJA_CAC`     | Si el cliente ha contactado con atención al cliente por alguna queja. |
| `COSTE_VENTA`   | Suma de descuentos aplicados en la venta. |
| `km_anno`       | Kilometraje medio recorrido por el coche al año. |
| `Mas_1_coche`   | Indica si el cliente ha comprado más de un coche. |
| `Revisiones`    | Número de revisiones pasadas del coche. |
| `Edad_Cliente`  | Edad actual del cliente. |
| `Tiempo`        | Años transcurridos entre compras de coches por el cliente. |



In [148]:
df.describe(include='all')

,CODE,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
count,58049,58049,58049,58049,58049,58049,58049,57159,57189,58049,...,44871,58049,58048,58049,58049.000000,58049.000000,58049,58049.000000,58049.000000,58049.000000
unique,58049,11,8,2,3,2,4,4,2,3,...,4,2,4,2,NaN,NaN,2,NaN,NaN,NaN
top,CLI58048,B,TIPO1,FUEL 2,Media,M,Contado,CASADO,M,Empresa,...,Alto,NO DATA,No,NO,NaN,NaN,False,NaN,NaN,NaN
freq,1,15993,23387,33085,39906,52969,30190,43529,40170,53262,...,17152,32011,29935,36268,NaN,NaN,41107,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2540.125756,11805.893607,NaN,3.529587,47.351255,1.859119
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1603.556511,10205.363971,NaN,2.528413,11.239984,3.091309
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,18.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1595.000000,0.000000,NaN,1.000000,40.000000,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2353.000000,11486.000000,NaN,3.000000,48.000000,0.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3301.000000,17927.000000,NaN,5.000000,56.000000,4.000000


### Limpieza de datos.

El dataset cuenta con un total de 24 columnas que debemos analizar para determinar cuáles serán relevantes en nuestro estudio. Para ello, es importante comprender el significado de cada una de ellas y su posible impacto en el análisis.

#### Manejo de filas duplicadas.

En caso de haber filas duplicadas, deberemos de eliminar estas para que no afecten al análisis posterior.

In [149]:
df.duplicated().sum() # Columnas duplicadas

np.int64(0)

In [150]:
df.drop_duplicates(inplace=True)

#### Manejo de filas nulas.

In [151]:
df.isnull().sum() # Valores nulos

CODE                   0
PRODUCTO               0
TIPO_CARROCERIA        0
COMBUSTIBLE            0
Potencia               0
TRANS                  0
FORMA_PAGO             0
ESTADO_CIVIL         890
GENERO               860
OcupaciOn              0
PROVINCIA              0
Campanna1              0
Campanna2              0
Campanna3              0
Zona_Renta         13178
REV_Garantia           0
Averia_grave           1
QUEJA_CAC              0
COSTE_VENTA            0
km_anno                0
Mas_1_coche            0
Revisiones             0
Edad_Cliente           0
Tiempo                 0
dtype: int64

In [152]:
df.shape[0]

58049

In [153]:
df['Zona_Renta'] = df['Zona_Renta'].fillna(df['Zona_Renta'].mode()[0])


In [154]:
df.isnull().sum() # Valores nulos

CODE                 0
PRODUCTO             0
TIPO_CARROCERIA      0
COMBUSTIBLE          0
Potencia             0
TRANS                0
FORMA_PAGO           0
ESTADO_CIVIL       890
GENERO             860
OcupaciOn            0
PROVINCIA            0
Campanna1            0
Campanna2            0
Campanna3            0
Zona_Renta           0
REV_Garantia         0
Averia_grave         1
QUEJA_CAC            0
COSTE_VENTA          0
km_anno              0
Mas_1_coche          0
Revisiones           0
Edad_Cliente         0
Tiempo               0
dtype: int64

### Selección de variables categóricas y continuas.


Para realizar correctamente un ánalisis de datos debemos de distinguir entre las variables categóricas, variables que representan diferentes categorías o grupos, y continuas, variables que pueden tomar cualquier valor dentro de un rango determinado.

In [155]:
v_continuas = []
v_categoricas = []

for col in df.columns:
    if df[col].dtype in ['int64', 'float64', 'bool']:  # Variables numéricas y booleanas como continuas
        v_continuas.append(col)
    else:
        v_categoricas.append(col)

print('Variables continuas:', ', '.join(v_continuas))
print('Variables categóricas:', ', '.join(v_categoricas))


Variables continuas: COSTE_VENTA, km_anno, Mas_1_coche, Revisiones, Edad_Cliente, Tiempo
Variables categóricas: CODE, PRODUCTO, TIPO_CARROCERIA, COMBUSTIBLE, Potencia, TRANS, FORMA_PAGO, ESTADO_CIVIL, GENERO, OcupaciOn, PROVINCIA, Campanna1, Campanna2, Campanna3, Zona_Renta, REV_Garantia, Averia_grave, QUEJA_CAC


Variables continuas (v_continuas):

Columnas numéricas (int64, float64), ya que representan cantidades medibles.
También incluyo las booleanas (bool), ya que pueden ser tratadas como binarias (0/1).

Variables categóricas (v_categoricas):

Columnas de tipo object, que son datos no numéricos como nombres de productos, tipo de combustible, etc.

In [156]:
df[v_categoricas].nunique()

CODE               58049
PRODUCTO              11
TIPO_CARROCERIA        8
COMBUSTIBLE            2
Potencia               3
TRANS                  2
FORMA_PAGO             4
ESTADO_CIVIL           4
GENERO                 2
OcupaciOn              3
PROVINCIA             53
Campanna1              2
Campanna2              2
Campanna3              2
Zona_Renta             4
REV_Garantia           2
Averia_grave           4
QUEJA_CAC              2
dtype: int64

#### Tratamiento de variables categóricas.

Para el posterior modelo de clasificación, necesitamos codificar nuestras variables object. Para ello, realizaremos una codificación ordinal que consiste en asignar valores enteros a las categorias basándose en su posición o jerarquía en los datos.

In [157]:
df[v_categoricas].head(3)

,CODE,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,PROVINCIA,Campanna1,Campanna2,Campanna3,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC
0,CLI1,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,M,Empresa,Asturias,SI,NO,NO,Medio-Bajo,NO DATA,Averia muy grave,SI
1,CLI2,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,F,Empresa,Toledo,NO,NO,NO,Medio-Bajo,SI,No,NO
2,CLI3,A,TIPO1,FUEL 1,Baja,M,Otros,CASADO,M,Empresa,Lerida,NO,NO,NO,Medio,NO DATA,No,NO


In [158]:
df[v_categoricas].describe()

,CODE,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,PROVINCIA,Campanna1,Campanna2,Campanna3,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC
count,58049,58049,58049,58049,58049,58049,58049,57159,57189,58049,58049,58049,58049,58049,58049,58049,58048,58049
unique,58049,11,8,2,3,2,4,4,2,3,53,2,2,2,4,2,4,2
top,CLI58048,B,TIPO1,FUEL 2,Media,M,Contado,CASADO,M,Empresa,Madrid,NO,NO,NO,Alto,NO DATA,No,NO
freq,1,15993,23387,33085,39906,52969,30190,43529,40170,53262,10874,37466,51103,51075,30330,32011,29935,36268


**CODE.**

In [159]:

df['CODE'].unique()

array(['CLI1', 'CLI2', 'CLI3', ..., 'CLI57952', 'CLI58038', 'CLI58048'],
      shape=(58049,), dtype=object)

**PRODUCTO.**

In [160]:
df['PRODUCTO'].unique()

array(['A', 'B', 'C', 'H', 'J', 'D', 'I', 'E', 'F', 'K', 'G'],
      dtype=object)

In [161]:

label_encoder = LabelEncoder()
df['PRODUCTO_encoded'] = label_encoder.fit_transform(df['PRODUCTO'])


El siguiente código se ha agregado para lograr que las columnas codificadas se agreguen justo después de la columna original, de manera que sea más fácil ver a qué se le ha asignado el valor codificado.

In [162]:

cols = df.columns.tolist()
disclosure_index = cols.index('PRODUCTO')
cols.insert(disclosure_index + 1, cols.pop(cols.index('PRODUCTO_encoded')))
df = df[cols]

In [163]:

df.head(2)

,CODE,PRODUCTO,PRODUCTO_encoded,TIPO_CARROCERIA,COMBUSTIBLE,Potencia,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,0,TIPO1,FUEL 1,Baja,M,Contado,CASADO,M,...,Medio-Bajo,NO DATA,Averia muy grave,SI,2892,0,False,2,18,0
1,CLI2,A,0,TIPO1,FUEL 1,Baja,M,Contado,CASADO,F,...,Medio-Bajo,SI,No,NO,1376,7187,False,2,53,0


**TIPO_CARROCERIA.**

In [164]:
df['TIPO_CARROCERIA'].unique()

array(['TIPO1', 'TIPO6', 'TIPO4', 'TIPO7', 'TIPO8', 'TIPO2', 'TIPO3',
       'TIPO5'], dtype=object)

In [165]:
label_encoder = LabelEncoder()
df['TIPO_CARROCERIA_encoded'] = label_encoder.fit_transform(df['TIPO_CARROCERIA'])

In [166]:
cols = df.columns.tolist()
disclosure_index = cols.index('TIPO_CARROCERIA')
cols.insert(disclosure_index + 1, cols.pop(cols.index('TIPO_CARROCERIA_encoded')))
df = df[cols]

In [167]:
df.head(2)

,CODE,PRODUCTO,PRODUCTO_encoded,TIPO_CARROCERIA,TIPO_CARROCERIA_encoded,COMBUSTIBLE,Potencia,TRANS,FORMA_PAGO,ESTADO_CIVIL,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,0,TIPO1,0,FUEL 1,Baja,M,Contado,CASADO,...,Medio-Bajo,NO DATA,Averia muy grave,SI,2892,0,False,2,18,0
1,CLI2,A,0,TIPO1,0,FUEL 1,Baja,M,Contado,CASADO,...,Medio-Bajo,SI,No,NO,1376,7187,False,2,53,0


**COMBUSTIBLE.**

In [168]:
df['COMBUSTIBLE'].unique()

array(['FUEL 1', 'FUEL 2'], dtype=object)

In [169]:
label_encoder = LabelEncoder()
df['COMBUSTIBLE_encoded'] = label_encoder.fit_transform(df['COMBUSTIBLE'])

In [170]:
cols = df.columns.tolist()
disclosure_index = cols.index('COMBUSTIBLE')
cols.insert(disclosure_index + 1, cols.pop(cols.index('COMBUSTIBLE_encoded')))
df = df[cols]

In [171]:
df.head(2)

,CODE,PRODUCTO,PRODUCTO_encoded,TIPO_CARROCERIA,TIPO_CARROCERIA_encoded,COMBUSTIBLE,COMBUSTIBLE_encoded,Potencia,TRANS,FORMA_PAGO,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,0,TIPO1,0,FUEL 1,0,Baja,M,Contado,...,Medio-Bajo,NO DATA,Averia muy grave,SI,2892,0,False,2,18,0
1,CLI2,A,0,TIPO1,0,FUEL 1,0,Baja,M,Contado,...,Medio-Bajo,SI,No,NO,1376,7187,False,2,53,0


**Potencia.**

In [172]:
df['Potencia'].unique()

array(['Baja', 'Media', 'Alta'], dtype=object)

In [173]:
label_encoder = LabelEncoder()
df['Potencia_encoded'] = label_encoder.fit_transform(df['Potencia'])

In [174]:
cols = df.columns.tolist()
disclosure_index = cols.index('Potencia')
cols.insert(disclosure_index + 1, cols.pop(cols.index('Potencia_encoded')))
df = df[cols]

In [175]:
df.head(2)

,CODE,PRODUCTO,PRODUCTO_encoded,TIPO_CARROCERIA,TIPO_CARROCERIA_encoded,COMBUSTIBLE,COMBUSTIBLE_encoded,Potencia,Potencia_encoded,TRANS,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Medio-Bajo,NO DATA,Averia muy grave,SI,2892,0,False,2,18,0
1,CLI2,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Medio-Bajo,SI,No,NO,1376,7187,False,2,53,0


**TRANS.**

In [176]:
df['TRANS'].unique()

array(['M', 'A'], dtype=object)

In [177]:
label_encoder = LabelEncoder()
df['TRANS_encoded'] = label_encoder.fit_transform(df['TRANS'])

In [178]:
cols = df.columns.tolist()
disclosure_index = cols.index('TRANS')
cols.insert(disclosure_index + 1, cols.pop(cols.index('TRANS_encoded')))
df = df[cols]

In [179]:
df.head(2)

,CODE,PRODUCTO,PRODUCTO_encoded,TIPO_CARROCERIA,TIPO_CARROCERIA_encoded,COMBUSTIBLE,COMBUSTIBLE_encoded,Potencia,Potencia_encoded,TRANS,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Medio-Bajo,NO DATA,Averia muy grave,SI,2892,0,False,2,18,0
1,CLI2,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Medio-Bajo,SI,No,NO,1376,7187,False,2,53,0


**FORMA_PAGO.**

In [180]:
df['FORMA_PAGO'].unique()

array(['Contado', 'Otros', 'Financiera Marca', 'Financiera Banco'],
      dtype=object)

In [181]:
label_encoder = LabelEncoder()
df['FORMA_PAGO_encoded'] = label_encoder.fit_transform(df['FORMA_PAGO'])

In [182]:
cols = df.columns.tolist()
disclosure_index = cols.index('FORMA_PAGO')
cols.insert(disclosure_index + 1, cols.pop(cols.index('FORMA_PAGO_encoded')))
df = df[cols]

In [183]:
df.head(2)

,CODE,PRODUCTO,PRODUCTO_encoded,TIPO_CARROCERIA,TIPO_CARROCERIA_encoded,COMBUSTIBLE,COMBUSTIBLE_encoded,Potencia,Potencia_encoded,TRANS,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Medio-Bajo,NO DATA,Averia muy grave,SI,2892,0,False,2,18,0
1,CLI2,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Medio-Bajo,SI,No,NO,1376,7187,False,2,53,0


**ESTADO_CIVIL.**

In [184]:
df['ESTADO_CIVIL'].unique()

array(['CASADO', 'SOLTERO', 'OTROS', 'EN PAREJA', nan], dtype=object)

In [185]:
label_encoder = LabelEncoder()
df['ESTADO_CIVIL_encoded'] = label_encoder.fit_transform(df['ESTADO_CIVIL'])

In [186]:
cols = df.columns.tolist()
disclosure_index = cols.index('ESTADO_CIVIL')
cols.insert(disclosure_index + 1, cols.pop(cols.index('ESTADO_CIVIL_encoded')))
df = df[cols]

In [187]:
df.head(2)

,CODE,PRODUCTO,PRODUCTO_encoded,TIPO_CARROCERIA,TIPO_CARROCERIA_encoded,COMBUSTIBLE,COMBUSTIBLE_encoded,Potencia,Potencia_encoded,TRANS,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Medio-Bajo,NO DATA,Averia muy grave,SI,2892,0,False,2,18,0
1,CLI2,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Medio-Bajo,SI,No,NO,1376,7187,False,2,53,0


**GENERO.**

In [188]:
df['GENERO'].unique()

array(['M', 'F', nan], dtype=object)

In [189]:
label_encoder = LabelEncoder()
df['GENERO_encoded'] = label_encoder.fit_transform(df['GENERO'])

In [190]:
cols = df.columns.tolist()
disclosure_index = cols.index('GENERO')
cols.insert(disclosure_index + 1, cols.pop(cols.index('GENERO_encoded')))
df = df[cols]

In [191]:
df.head(2)

,CODE,PRODUCTO,PRODUCTO_encoded,TIPO_CARROCERIA,TIPO_CARROCERIA_encoded,COMBUSTIBLE,COMBUSTIBLE_encoded,Potencia,Potencia_encoded,TRANS,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Medio-Bajo,NO DATA,Averia muy grave,SI,2892,0,False,2,18,0
1,CLI2,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Medio-Bajo,SI,No,NO,1376,7187,False,2,53,0


**OcupaciOn.**

In [194]:
df['OcupaciOn'].unique()

array(['Empresa', 'Funcionario', 'Autonomo'], dtype=object)

In [195]:
label_encoder = LabelEncoder()
df['OcupaciOn_encoded'] = label_encoder.fit_transform(df['OcupaciOn'])

In [196]:
cols = df.columns.tolist()
disclosure_index = cols.index('OcupaciOn')
cols.insert(disclosure_index + 1, cols.pop(cols.index('OcupaciOn_encoded')))
df = df[cols]

In [197]:
df.head(2)

,CODE,PRODUCTO,PRODUCTO_encoded,TIPO_CARROCERIA,TIPO_CARROCERIA_encoded,COMBUSTIBLE,COMBUSTIBLE_encoded,Potencia,Potencia_encoded,TRANS,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Medio-Bajo,NO DATA,Averia muy grave,SI,2892,0,False,2,18,0
1,CLI2,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Medio-Bajo,SI,No,NO,1376,7187,False,2,53,0


**PROVINCIA.**

In [198]:
df['PROVINCIA'].unique()

array(['Asturias', 'Toledo', 'Lerida', 'Madrid', 'Santa Cruz de Tenerife',
       'Pontevedra', 'Lacoruna', 'Barcelona', 'Cordoba', 'Guipuzcua',
       'Valladolid', 'Castellon', 'Valencia', 'Las Palmas', 'La Rioja',
       'Baleares', 'Zaragoza', 'Alicante', 'Tarragona', 'Leon', 'Lugo',
       'Badajoz', 'Vizcaya', 'Sevilla', 'Guadalajara', 'Ciudad Real',
       'Cantabria', 'Orense', 'Navarra', 'Gerona', 'Malaga', 'Jaen',
       'Murcia', 'Burgos', 'Granada', 'Alava', 'Cuenca', 'Cadiz',
       'Salamanca', 'Albacete', 'Almeria', 'Teruel', 'Segovia',
       'Palencia', 'Huelva', 'Huesca', 'Zamora', 'Avila', 'Soria',
       'Caceres', 'Melilla', 'Ceuta', 'Francia'], dtype=object)

In [199]:
label_encoder = LabelEncoder()
df['PROVINCIA_encoded'] = label_encoder.fit_transform(df['PROVINCIA'])

In [200]:
cols = df.columns.tolist()
disclosure_index = cols.index('PROVINCIA')
cols.insert(disclosure_index + 1, cols.pop(cols.index('PROVINCIA_encoded')))
df = df[cols]

In [201]:
df.head(2)

,CODE,PRODUCTO,PRODUCTO_encoded,TIPO_CARROCERIA,TIPO_CARROCERIA_encoded,COMBUSTIBLE,COMBUSTIBLE_encoded,Potencia,Potencia_encoded,TRANS,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Medio-Bajo,NO DATA,Averia muy grave,SI,2892,0,False,2,18,0
1,CLI2,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Medio-Bajo,SI,No,NO,1376,7187,False,2,53,0


**Campanna1, Campanna2, Campanna3.**

In [202]:
df[['Campanna1', 'Campanna2', 'Campanna3']].apply(pd.Series.unique)

,Campanna1,Campanna2,Campanna3
0,SI,NO,NO
1,NO,SI,SI


In [203]:
label_encoder = LabelEncoder()

for col in ['Campanna1', 'Campanna2', 'Campanna3']:
    df[f'{col}_encoded'] = label_encoder.fit_transform(df[col])

In [204]:
label_encoder = LabelEncoder()

for col in ['Campanna1', 'Campanna2', 'Campanna3']:
    df[f'{col}_encoded'] = label_encoder.fit_transform(df[col])
    cols = df.columns.tolist()
    disclosure_index = cols.index(col)
    cols.insert(disclosure_index + 1, cols.pop(cols.index(f'{col}_encoded')))
    df = df[cols]

In [205]:
df.head(2)

,CODE,PRODUCTO,PRODUCTO_encoded,TIPO_CARROCERIA,TIPO_CARROCERIA_encoded,COMBUSTIBLE,COMBUSTIBLE_encoded,Potencia,Potencia_encoded,TRANS,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Medio-Bajo,NO DATA,Averia muy grave,SI,2892,0,False,2,18,0
1,CLI2,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Medio-Bajo,SI,No,NO,1376,7187,False,2,53,0


**Zona_Renta**                                

In [206]:
df['Zona_Renta'].unique()

array(['Medio-Bajo', 'Medio', 'Alto', 'Otros'], dtype=object)

In [207]:
label_encoder = LabelEncoder()
df['Zona_Renta_encoded'] = label_encoder.fit_transform(df['Zona_Renta'])

In [208]:
cols = df.columns.tolist()
disclosure_index = cols.index('Zona_Renta')
cols.insert(disclosure_index + 1, cols.pop(cols.index('Zona_Renta_encoded')))
df = df[cols]

In [209]:
df.head(2)

,CODE,PRODUCTO,PRODUCTO_encoded,TIPO_CARROCERIA,TIPO_CARROCERIA_encoded,COMBUSTIBLE,COMBUSTIBLE_encoded,Potencia,Potencia_encoded,TRANS,...,Zona_Renta_encoded,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,2,NO DATA,Averia muy grave,SI,2892,0,False,2,18,0
1,CLI2,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,2,SI,No,NO,1376,7187,False,2,53,0


**REV_Garantia**

In [210]:
df['REV_Garantia'].unique()

array(['NO DATA', 'SI'], dtype=object)

In [211]:
label_encoder = LabelEncoder()
df['REV_Garantia_encoded'] = label_encoder.fit_transform(df['REV_Garantia'])

In [212]:
cols = df.columns.tolist()
disclosure_index = cols.index('REV_Garantia')
cols.insert(disclosure_index + 1, cols.pop(cols.index('REV_Garantia_encoded')))
df = df[cols]

In [213]:
df.head(2)

,CODE,PRODUCTO,PRODUCTO_encoded,TIPO_CARROCERIA,TIPO_CARROCERIA_encoded,COMBUSTIBLE,COMBUSTIBLE_encoded,Potencia,Potencia_encoded,TRANS,...,REV_Garantia,REV_Garantia_encoded,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,NO DATA,0,Averia muy grave,SI,2892,0,False,2,18,0
1,CLI2,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,SI,1,No,NO,1376,7187,False,2,53,0


**Averia_grave**

In [214]:
df['Averia_grave'].unique()

array(['Averia muy grave', 'No', 'Averia leve', 'Averia grave', nan],
      dtype=object)

In [215]:
label_encoder = LabelEncoder()
df['Averia_grave_encoded'] = label_encoder.fit_transform(df['Averia_grave'])

In [216]:
cols = df.columns.tolist()
disclosure_index = cols.index('Averia_grave')
cols.insert(disclosure_index + 1, cols.pop(cols.index('Averia_grave_encoded')))
df = df[cols]

In [217]:
df.head(2)

,CODE,PRODUCTO,PRODUCTO_encoded,TIPO_CARROCERIA,TIPO_CARROCERIA_encoded,COMBUSTIBLE,COMBUSTIBLE_encoded,Potencia,Potencia_encoded,TRANS,...,REV_Garantia_encoded,Averia_grave,Averia_grave_encoded,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,0,Averia muy grave,2,SI,2892,0,False,2,18,0
1,CLI2,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,1,No,3,NO,1376,7187,False,2,53,0


**QUEJA_CAC**

In [218]:
df['QUEJA_CAC'].unique()

array(['SI', 'NO'], dtype=object)

In [219]:
label_encoder = LabelEncoder()
df['QUEJA_CAC_encoded'] = label_encoder.fit_transform(df['QUEJA_CAC'])

In [220]:
cols = df.columns.tolist()
disclosure_index = cols.index('QUEJA_CAC')
cols.insert(disclosure_index + 1, cols.pop(cols.index('QUEJA_CAC_encoded')))
df = df[cols]

In [221]:
df.head(2)

,CODE,PRODUCTO,PRODUCTO_encoded,TIPO_CARROCERIA,TIPO_CARROCERIA_encoded,COMBUSTIBLE,COMBUSTIBLE_encoded,Potencia,Potencia_encoded,TRANS,...,Averia_grave,Averia_grave_encoded,QUEJA_CAC,QUEJA_CAC_encoded,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,CLI1,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,Averia muy grave,2,SI,1,2892,0,False,2,18,0
1,CLI2,A,0,TIPO1,0,FUEL 1,0,Baja,1,M,...,No,3,NO,0,1376,7187,False,2,53,0


#### Tratamiento de variables continuas.

In [222]:
df[v_continuas].nunique()

COSTE_VENTA      2833
km_anno         21512
Mas_1_coche         2
Revisiones         14
Edad_Cliente       54
Tiempo             14
dtype: int64

In [223]:
df[v_continuas].head(3)

,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
0,2892,0,False,2,18,0
1,1376,7187,False,2,53,0
2,1376,0,True,4,21,3


In [224]:
df[v_continuas].describe()

,COSTE_VENTA,km_anno,Revisiones,Edad_Cliente,Tiempo
count,58049.000000,58049.000000,58049.000000,58049.000000,58049.000000
mean,2540.125756,11805.893607,3.529587,47.351255,1.859119
std,1603.556511,10205.363971,2.528413,11.239984,3.091309
min,0.000000,0.000000,0.000000,18.000000,0.000000
25%,1595.000000,0.000000,1.000000,40.000000,0.000000
50%,2353.000000,11486.000000,3.000000,48.000000,0.000000
75%,3301.000000,17927.000000,5.000000,56.000000,4.000000
max,18455.000000,182331.000000,13.000000,71.000000,14.000000
